V tomto notebooku si ukážeme, jak jednoduše načíst marcový dokument pomocí knihovny pymarc, jak marcový záznam vypadá, jak data z marcového dokumentu vytáhnout a uložit do csv tabulky. Nakonec z dat zjistíme nejčetnějšího autora za každý rok.   


#### Příprava 
Neprve si data stáhneme a podíváme se, co je uvnitř. Data jsou uložena v knihovním formátu MARC s koncovnkou .mrc. Pro práci budeme používat knihovnu pymarc, která dokáže marcový formát jednodušše načíst.   
Nejprve si vypíšeme, jak takový záznam vypadá.  

In [9]:
from pymarc import MARCReader

# 'data/csv/ucla_B.csv'
# 'data/csv/ucla_ret.csv'
# 'data/csv/ucla_smz.csv'
# 'data/csv/ucla_int.csv'
# 'data/csv/ucla_cle.csv'
# 'data/csv/ucla_trl.csv'

# Cesta k marcovemu dokumentu
database = 'data/ucla/ucla_cle.mrc'

# Otevreni souboru
with open(database, 'rb') as data:
    # Nacteni marcu
    reader = MARCReader(data)

    # Iterace skrz nacteny soubor
    for record in reader:
        # Vypsani marcovych zaznamu
        print(record)

        # Chceme vypsat pouze jeden zaznam, tak po prvnim vypsani zavolame funkci break
        break


=LDR  00875nab a22003134a 4500
=001  001457852
=003  CZ\PrUCL
=005  20220321104606.0
=008  090923e197808\\xr\\\\\\\\\\\\|||\||cze\d
=035  \\$a(ISIS-B70-MFN)78971
=035  \\$a(ISIS-B70-ID)785001
=040  \\$aABB060$bcze
=080  \\$a7.01$2MRF
=080  \\$a(0:82-4)$2MRF
=100  1\$aKliment, Alexandr,$d1929-2017$7jk01060558$4aut
=245  10$aKultura je když... /$cAlexandr Kliment.
=500  \\$aPodnázev zdrojového dokumentu: (London, Index on Censorship).
=520  2\$aÚvaha.
=599  \\$aCLB-CPK
=650  07$aestetika$7ph117189$2czenas
=655  \7$aúvahy$7fd134000$2czenas
=773  0\$tSpektrum$gR. 1978, č. 1, srpen, s. 77-79$q1978:1<77$9197808
=964  \\$aB70
=964  \\$aSMZ
=964  \\$aCLE
=OWN  \\$aUCLA
=SIF  \\$afk
=910  \\$aABB060
=ZAZ  \\$d197808$s1978$z1$l77



Vidíme, že marcový soubor má jasnou strukuru. Má několik polí, která jsou označena zpravidla třemi číslicem, případně třemi písmeny. Každý kód má svou vnitřní logiku, např. pole pro věcné popisy vždy začínají číslicí 6XX. 
Za číslem - tagem pole se obvyke nachází dva indikátory. Pokud indikátor není definovaný, píše se místo něj zpětné lomítko (\\). 
Většina polí se dělí na podpole. Nachází se za dolarem ($) a označují se buď jedním písmenem, nebo číslicí.  



In [10]:
# Otevreni souboru
with open(database, 'rb') as data:
    # Nacteni marcu
    reader = MARCReader(data)

    # Iterace skrz nacteny soubor
    for record in reader:
        # Vypsani marcovych zaznamu
        # K nekterym zaznamum muzeme pristupovat pres teckovou notaci, tedy record.leader nebo record.title 
        print("Záznam: " + record.leader)
        
        # Pred vypsanim je standardem se nejprve podivat, zda zaznam existuje (tedy ze neni None). 
        # Pokud bychom se totiz snazili vypsat None zaznam, kod by vyhodil error. 
        if record.title is not None:
            print("Nazev: " + record.title)
        if record.author is not None:
            print("Autor: " + record.author)
        # Pokud chceme zjistit, zda pole ktere nema teckovou notaci neni None, je potreba zavolat funci .get_field(), pripadne .get_field()     
        if record.get_fields('655') is not None:    
            # K polim, ktere nemaji teckovou notaci, se pristupuje pres zavorky 
            print("Žánr: " + record['655']['a'])
        break        

Záznam: 00875nab a22003134a 4500
Nazev: Kultura je když... /
Autor: Kliment, Alexandr, 1929-2017 jk01060558 aut
Žánr: úvahy


Také by nás mohlo zajímat, kolik záznamů je v dané databázi. K tomu si vytvoříme samostatnou funkci, kterou pak jen zavoláme.

In [11]:
# Vlastni funkce definujeme pomoci slova def
def number_of_records(database):
    with open(database, 'rb') as data:
    # Nacteni marcu
        reader = MARCReader(data)
        # Vytvorime counter, ke kteremu pri kazdem zaznamu pricteme jednicku
        counter = 0
        # Vzhledem k tomu, ze record nepotrebujeme, tak muzeme pouzit podtrzitko (_), ktere hodnotu bude ignorovat 
        for _ in reader:
            counter += 1
    return counter 

print("V databazi je " + str(number_of_records(database)) + " zaznamu.")        

V databazi je 9612 zaznamu.


S marcovým dokumentem se nepracuje příliš dobře, proto je lepší si data uložit do jednodušší tabulky. V této fázi si musíme ujasnit, jaká data budeme chtít. V našem příkladu si budeme chtít uložit název, autora, autorův kód, rok vydání a pak pole '600 $a','650 $a', '655 $a' a '773 $t'. 
Veškeré záznamy začínající číslem 6XX jsou věcné údaje o záznamu. Tato pole se mohou opakovat.     
Pod polem '600' se skrývají osoby, o kterých záznam je nebo případně osoby, kterým je záznam dedikován. 
Pod polem '650' se nacházejí věcné termíny/téma, tzn. o čem záznam je.
Pod polem '655' pak najdeme žánr daného záznamu. Na rozdíl od polí '600' a '650' by pole '655' mělo být přítomné u každého záznamu.
Záznamy začínající čísly 76X - 78X se nazývají propojovací pole a slouží pro zápis zdrojového (773) nebo recenzovaného (787) dokumentu. 


Pro ukládání máme připravenou funkci save_to_dict(record, dict, field_list), která nám jeden záznam(record) uloží do slovníku (struktury v pythonu) dict. Není strategické si do tabulky ukládat všechna pole, podpole a indikátory, proto funkci save_to_dict kromě záznamu předáme také seznam polí field_list, která chceme uložit. Seznam field_list sestává z tuplů (kolekce v jazyce Python), kde každý tuple vypadá následovně. Na první pozici je název klíče, pod kterým bude pole uloženo, na druhé pozici tag pole a na třetí tag podpole, např. ('author', '100', 'a'). 

In [12]:
def save_to_dict(record, dict, field_list):
    if not record is None:
        try:
            # Iterace skrz tuples v seznamu field_list
            for field_tags in field_list:
                # Nazev klice ve slovniku
                dict_key_name =  field_tags[0]

                # Tag pole
                tag =  field_tags[1]

                # Tag podpole
                subfield_tag =  field_tags[2]
                
                # Seznam do ktereho pridame hodnoty a nasledne pridame do slovniku
                dict_add_list = []
                
                # Iterace pres vsechna pole s tagem 'tag'
                for field in record.get_fields(tag):
                    
                    # Pokud pole nema zadna podpole, pridame cele pole do listu dict_add_list
                    if subfield_tag is None:
                        dict_add_list.append(str(field))
                    
                    # Pokud subtag je instance slice, tedy to znamena, ze chceme jen nejakou cast pole, ktera neni definovana subpolem,
                    # pridame cast pole do slovniku dict_add_list    
                    elif isinstance(subfield_tag, slice):
                        dict_add_list.append(str(field) [subfield_tag])     
                    
                    # Pokud pole obsahuje podpole, pridame do slovniku dict_add_list jen podpole
                    elif '$'+subfield_tag in str(field):  
                        dict_add_list.append(str(field[subfield_tag]))

                # Do klice z tuplu pridame cely seznam dict_add_list         
                dict[dict_key_name].append(dict_add_list)
        except Exception as error:
            print("Exception: " + type(error).__name__)  
            print("964 Field: " + str(record.get_fields('964')))  
            print("LDR: " + str(record.leader))   
    return dict 

In [13]:
# Prikaz ktery naistaluje knihovny
%pip install pandas 

import re
import pandas as pd

# Z cesty vytahneme typ databaze
pattern = r"data/ucla/ucla_(.*?)\.mrc"

# Najdeme substring pomoci regularniho vyrazu
database_type = re.search(pattern, database).group(1)

out = 'data/csv/out_{}.csv'.format(database_type)

with open(database, 'rb') as data:
    reader = MARCReader(data)
    # Seznam poli, ktere si chceme ulozit
    field_list = [('title', '245', 'a'),
                ('author', '100', 'a'),
                ('author code', '100', '7'),
                # Rok je schovany v poli 008 na 13. az 16. miste, proto vyuzijeme funkci slice
                ('year', '008', slice(13,17, None)),
                ('figures', '600', 'a'),
                ('description', '650', 'a'),
                ('genre', '655', 'a'),
                ('magazine', '773', 't')]
    dict = {}
    for t in field_list:
        dict_key_name = t[0]
        dict[dict_key_name] = []
    for record in reader:
        dict = save_to_dict(record, dict, field_list)
    df = pd.DataFrame.from_dict(dict)

    # U jmen si chceme ulozit jmeno a prijmeni bez koncove carky ',', ktera je na konci stringu
    df['figures'] = df['figures'].apply(lambda x: [y[:y.rfind(',')] if isinstance(y, str) and len(y) > 0 else y for y in x]) 
    df['author'] = df['author'].apply(lambda x: [y[:y.rfind(',')] if isinstance(y, str) and len(y) > 0 else y for y in x])  

    # Aby se nam list hodnot lepe ukladal, vytvorime z listu jeden string a jednotlive elementy spojime strednikem ';' 
    for column in df.columns:
        df[column] = df[column].apply(lambda x: ';'.join(x))
    df.to_csv(out, encoding = 'utf8', sep = ",")     


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


Teď, když máme hodnoty uložené v csv, bude mnohem jednodušší s nimi pracovat. Uložená data načteme pomocí knihovny pandas do datové struktury DataFrame (která je podobná např. excelovské tabulce). Řádky v DataFramu reprezentují jednotlivé záznamy, sloupce pak jeden typ (např. jmeno autora).
Některá pole a podpole mohou opakovat, ty jsou  v csv spojené středníkem. V DataFramu je pak podpole rozpojíme a převede do listu (seznamu). POZOR: Kvůli obecnosti jsou v listu uloženy veškeré hodnoty. Je jednodušší, když ve sloupci je pouze jeden datový typ (např. list nebo string), než když jsou některé hodnoty v listu a některé jsou pouze string. Pokud jsme si jisti, že v původních záznamech se pole nikde neopakovalo, můžeme celý sloupec převést do jiného formátu.  

In [14]:
import pandas as pd

out = 'data/csv/out_{}.csv'.format(database_type)

# Cesta k nasim datum
csv_data = out

# Nacteni dat
df = pd.read_csv(csv_data, delimiter=',')
# Odstraneni zbytecneho sloupce
df = df.drop(['Unnamed: 0'], axis = 1)

for column in df.columns:
    if df[column].dtype != 'int64':   
        # Hodnoty spojene v jeden string zpatky rozdelime do listu, aby se nam s nim lepe pracovalo
        df[column] = df[column].apply(lambda x: x.split(';') if isinstance(x, str)  else [])


Nakonec si ukážeme, jak z dat zjistit, který autor v jakém roce publikoval nejvíce článků (tedy se v záznamech v daném roce nejčastěji objevuje jako autor). 
Tím, že máme data uložena do DataFramu, umožňuje nám to využít funkce knihovny pandas. V našem případě využijeme funkci groupby, která data seskupí podle vybraného sloupce. 
Použijeme také funkci Counter, která nám spočte výskyt každého elementu.
V neposlední řadě využijeme anonymní funkci lambda.

Kód je napsán tak, aby se parametry daly jednoduše upravit. Můžeme si tak zkusit změnit sloupec picked_colum na 'figures', čímž zjistíme, o kom se nejvíce daný rok psalo.

In [15]:
from collections import Counter

# Vybereme sloupec, ktery chceme vypsat
picked_column = 'author'

# Data ocistime o prazdne hodnoty
data_filtered = df[df[picked_column].apply(lambda x: len(x) > 0)]

# Data seskupime podle nejcastejsiho elementu v kazde hodnote groupby
most_common = data_filtered.groupby('year')[picked_column].apply(lambda x: Counter(element for elements in x for element in elements).most_common(1)[0][0] if len(x) > 0 else None) 

# Nejcastejsi hodnoty vypiseme
print(most_common)        


year
1948      Jandáček, Antonín
1949      Dresler, Jaroslav
1950          Demetz, Peter
1951          Demetz, Peter
1952        Peška, Vladimír
1953          Vlach, Robert
1954        Hostovský, Egon
1955          Vlach, Robert
1956         Želivan, Pavel
1957          Vlach, Robert
1958              Den, Petr
1959              Den, Petr
1960      Dresler, Jaroslav
1961              Den, Petr
1962              Den, Petr
1963              Den, Petr
1964              Den, Petr
1965              Den, Petr
1966              Den, Petr
1967              Den, Petr
1968              Den, Petr
1969              Den, Petr
1970              Den, Petr
1971    Kratochvil, Antonín
1972    Kratochvil, Antonín
1973    Kratochvil, Antonín
1974      Dresler, Jaroslav
1975    Kratochvil, Antonín
1976      Dresler, Jaroslav
1977      Dresler, Jaroslav
1978      Dresler, Jaroslav
1979      Dresler, Jaroslav
1980           Kovtun, Jiří
1981    Kratochvil, Antonín
1982    Kratochvil, Antonín
1983      Dresl